In [1]:
import sys

sys.path.insert(0, '..')
import os
from basico import *
from process_bigraph import Composite, pf
from builder import Builder, Process as builderProcess, ProcessTypes
from biosimulator_processes.processes.copasi_process import CopasiProcess
from biosimulator_processes.process_types import MODEL_TYPE, CHANGES_SCHEMA
from biosimulator_processes.utils import play_composition

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


In [2]:
MODEL_TYPE

{'model_id': 'string',
 'model_source': 'string',
 'model_language': {'_type': 'string', '_default': 'sbml'},
 'model_name': {'_type': 'string', '_default': 'composite_process_model'},
 'model_changes': {'species_changes': 'tree[string]',
  'global_parameter_changes': 'tree[string]',
  'reaction_changes': 'tree[string]'},
 'model_units': 'tree[string]'}